In [1]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import Birch

import pandas as pd
import numpy as np

In [2]:
import requests
import pandas as pd

# Tu token de acceso
access_token = ''

# ID del archivo que deseas leer de Google Drive
file_id = '16f1oPpO_vQSLNEmnQJMOa7lee5c49Uo9'

# URL de la API de Google Drive para descargar el archivo
url = f'https://www.googleapis.com/drive/v3/files/{file_id}?alt=media'

# Encabezados de la solicitud
headers = {
    'Authorization': f'Bearer {access_token}'
}

# Realizar la solicitud GET para descargar el archivo
response = requests.get(url, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Guardar el contenido en un archivo temporal
    with open('temp_file.csv', 'wb') as f:
        f.write(response.content)
    
    # Leer el archivo en un DataFrame de pandas
    df = pd.read_csv('temp_file.csv')
    
    # Visualizar los primeros registros del DataFrame
    print(df.head())
else:
    print(f'Error al descargar el archivo: {response.status_code}')


   p_id  age v_o_id sex  K73  H35  I65  J45  F30-F39  E66  ...  C40-C41  C53  \
0    11   77   11_5   F  NaN  NaN  NaN  NaN      NaN  NaN  ...      NaN  NaN   
1    23   63   23_4   U  NaN  NaN  NaN  NaN      NaN  1.0  ...      NaN  NaN   
2    39   49   39_5   F  NaN  NaN  NaN  NaN      NaN  NaN  ...      NaN  NaN   
3    43   52   43_5   F  NaN  NaN  NaN  NaN      NaN  NaN  ...      NaN  NaN   
4    59   78   59_6   F  NaN  NaN  NaN  NaN      NaN  NaN  ...      NaN  NaN   

   Z89  C43  C46  C56  C73  pred1y_E10-E14  years_with_diabetes  \
0  NaN  NaN  NaN  NaN  NaN             1.0                    0   
1  NaN  NaN  NaN  NaN  NaN             1.0                    0   
2  NaN  NaN  NaN  NaN  NaN             1.0                    0   
3  NaN  NaN  NaN  NaN  NaN             NaN                    0   
4  NaN  NaN  NaN  NaN  NaN             NaN                    0   

        stratification  
0  predicted_condition  
1  predicted_condition  
2  predicted_condition  
3         no_con

In [3]:
def obtener_mejor_numero_clusters(clustering_method, data, max_clusters=10):
    mejor_score = -1
    mejor_numero_clusters = 0

    for n_clusters in range(2, max_clusters+1):
        clustering = clustering_method(n_clusters=n_clusters)

        clustering.fit(data)

        silhouette_avg = silhouette_score(data, clustering.labels_)

        if silhouette_avg > mejor_score:
            mejor_score = silhouette_avg
            mejor_numero_clusters = n_clusters

    return mejor_numero_clusters

In [5]:
import pandas as pd
df = df.fillna(0)
df.loc[df['sex'] == 'F', 'sex'] = 2
df.loc[df['sex'] == 'M', 'sex'] = 1
df.loc[df['sex'] == 'U', 'sex'] = 3
df = df.drop(columns = ['p_id', 'v_o_id', 'stratification'], axis= 1)
df = np.array(df)

In [6]:
clustering_method = Birch

data = df[:20]

mejor_numero_clusters = obtener_mejor_numero_clusters(clustering_method, data)

print("Mejor número de clusters según Silhouette:", mejor_numero_clusters)


Mejor número de clusters según Silhouette: 3


In [7]:

def k_means(df, lista_k):
  #df = np.array(db)
  modelos = []
  for k in lista_k:
    # Enternamiento
    km = KMeans(n_clusters=k, random_state=37)
    km.fit(df)
    modelos.append(km)
  return modelos

def metrics(df, l_modelos):
  scores = {'davie_bouldin' : [], 'calinski_harabasz': [], 'sillhouette': []}
  for m in  l_modelos:
    scores['davie_bouldin'].append(davies_bouldin_score(df, m.labels_))
    scores['calinski_harabasz'].append(calinski_harabasz_score(df, m.labels_))
    scores['sillhouette'].append(silhouette_score(df, m.labels_))
  return scores

def informe_metrics(scores, lista_k):
    df = pd.DataFrame(scores)
    df.insert(0, 'numero de k', lista_k)
    #df['davie_nom'] = df['davie_bouldin'].apply(lambda a: 1 / (a + 1))
    #df['calinski_nom'] = df['calinski_harabasz'].apply(lambda a: 1 - (1/ (a + 1)))
    #df['sillhoutte_nom'] = df['sillhouette'].apply(lambda a: (a + 1)/ 2)

    # Calcular la media de las últimas tres métricas
    #df['media metrics'] = df.iloc[:, -3:].sum(axis=1) / 3
    df.to_csv("scores.xlsx", index=False)
    return df


def grafics():
  pass

def main(db):
  df = np.array(db)
  k = [i for i in range(2, 41)]
  modelos = k_means(df, k)
  scores = metrics(df, modelos)
  informe_metrics(scores, k)

  return (modelos, scores)
